<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Artificial Intelligence in Finance

## Normative Finance

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Uncertainty and Risk

In [ ]:
import numpy as np

In [ ]:
S0 = 10
B0 = 10

In [ ]:
S1 = np.array((20, 5))
B1 = np.array((11, 11))

In [ ]:
M0 = np.array((S0, B0))
M0

In [ ]:
M1 = np.array((S1, B1)).T
M1

In [ ]:
K = 14.5

In [ ]:
C1 = np.maximum(S1 - K, 0)
C1

In [ ]:
phi = np.linalg.solve(M1, C1)
phi

In [ ]:
np.allclose(C1, np.dot(M1, phi))

In [ ]:
C0 = np.dot(M0, phi)
C0

## Expected Utility Theory

In [ ]:
def u(x):
    return np.sqrt(x)

In [ ]:
phi_A = np.array((0.75, 0.25))
phi_D = np.array((0.25, 0.75))

In [ ]:
np.dot(M0, phi_A) == np.dot(M0, phi_D)

In [ ]:
A1 = np.dot(M1, phi_A)
A1

In [ ]:
D1 = np.dot(M1, phi_D)
D1

In [ ]:
P = np.array((0.5, 0.5))

In [ ]:
def EUT(x):
    return np.dot(P, u(x))

In [ ]:
EUT(A1)

In [ ]:
EUT(D1)

In [ ]:
from scipy.optimize import minimize

In [ ]:
w = 10

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: np.dot(M0, phi) - w}

In [ ]:
def EUT_(phi):
    x = np.dot(M1, phi)
    return EUT(x)

In [ ]:
opt = minimize(lambda phi: -EUT_(phi),
               x0=phi_A,
               constraints=cons)

In [ ]:
opt

In [ ]:
EUT_(opt['x'])

In [ ]:
np.dot(M0, opt['x'])

## Mean-Variance Portfolio Theory

In [ ]:
rS = S1 / S0 - 1
rS

In [ ]:
rB = B1 / B0 - 1
rB

In [ ]:
def mu(rX):
    return np.dot(P, rX)

In [ ]:
mu(rS)

In [ ]:
mu(rB)

In [ ]:
rM = M1 / M0 - 1
rM

In [ ]:
mu(rM)

In [ ]:
def var(rX):
    return ((rX - mu(rX)) ** 2).mean()

In [ ]:
var(rS)

In [ ]:
var(rB)

In [ ]:
def sigma(rX):
    return np.sqrt(var(rX))

In [ ]:
sigma(rS)

In [ ]:
sigma(rB)

In [ ]:
np.cov(rM.T, aweights=P, ddof=0)

In [ ]:
phi = np.array((0.5, 0.5))

In [ ]:
def mu_phi(phi):
    return np.dot(phi, mu(rM))

In [ ]:
mu_phi(phi)

In [ ]:
def var_phi(phi):
    cv = np.cov(rM.T, aweights=P, ddof=0)
    return np.dot(phi, np.dot(cv, phi))

In [ ]:
var_phi(phi)

In [ ]:
def sigma_phi(phi):
    return var_phi(phi) ** 0.5

In [ ]:
sigma_phi(phi)

In [ ]:
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'

In [ ]:
phi_mcs = np.random.random((2, 200))

In [ ]:
phi_mcs = (phi_mcs / phi_mcs.sum(axis=0)).T

In [ ]:
mcs = np.array([(sigma_phi(phi), mu_phi(phi))
                for phi in phi_mcs])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(mcs[:, 0], mcs[:, 1], 'ro')
plt.xlabel('expected volatility')
plt.ylabel('expected return');

In [ ]:
P = np.ones(3) / 3
P

In [ ]:
S1 = np.array((20, 10, 5))

In [ ]:
T0 = 10
T1 = np.array((1, 12, 13))

In [ ]:
M0 = np.array((S0, T0))
M0

In [ ]:
M1 = np.array((S1, T1)).T
M1

In [ ]:
rM = M1 / M0 - 1
rM

In [ ]:
mcs = np.array([(sigma_phi(phi), mu_phi(phi))
                for phi in phi_mcs])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(mcs[:, 0], mcs[:, 1], 'ro')
plt.xlabel('expected volatility')
plt.ylabel('expected return');

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}

In [ ]:
bnds = ((0, 1), (0, 1))

In [ ]:
min_var = minimize(sigma_phi, (0.5, 0.5),
                   constraints=cons, bounds=bnds)

In [ ]:
min_var

In [ ]:
def sharpe(phi):
    return mu_phi(phi) / sigma_phi(phi)

In [ ]:
max_sharpe = minimize(lambda phi: -sharpe(phi), (0.5, 0.5),
               constraints=cons, bounds=bnds)

In [ ]:
max_sharpe

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(mcs[:, 0], mcs[:, 1], 'ro', ms=5)
plt.plot(sigma_phi(min_var['x']), mu_phi(min_var['x']),
         '^', ms=12.5, label='minimum volatility')
plt.plot(sigma_phi(max_sharpe['x']), mu_phi(max_sharpe['x']),
         'v', ms=12.5, label='maximum Sharpe ratio')
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.legend();

In [ ]:
cons = [{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1},
       {'type': 'eq', 'fun': lambda phi: mu_phi(phi) - target}]

In [ ]:
bnds = ((0, 1), (0, 1))

In [ ]:
targets = np.linspace(mu_phi(min_var['x']), 0.16)

In [ ]:
frontier = []
for target in targets:
    phi_eff = minimize(sigma_phi, (0.5, 0.5),
                       constraints=cons, bounds=bnds)['x']
    frontier.append((sigma_phi(phi_eff), mu_phi(phi_eff)))
frontier = np.array(frontier)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(frontier[:, 0], frontier[:, 1], 'mo', ms=5,
         label='efficient frontier')
plt.plot(sigma_phi(min_var['x']), mu_phi(min_var['x']),
         '^', ms=12.5, label='minimum volatility')
plt.plot(sigma_phi(max_sharpe['x']), mu_phi(max_sharpe['x']),
         'v', ms=12.5, label='maximum Sharpe ratio')
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.legend();

## Capital Asset Pricing Model

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot((0, 0.3), (0.01, 0.22), label='capital market line')
plt.plot(0, 0.01, 'o', ms=9, label='risk-less asset')
plt.plot(0.2, 0.15, '^', ms=9, label='market portfolio')
plt.annotate('$(0, \\bar{r})$', (0, 0.01), (-0.01, 0.02))
plt.annotate('$(\sigma_M, \mu_M)$', (0.2, 0.15), (0.19, 0.16))
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.legend();

In [ ]:
phi_M = np.array((0.8, 0.2))

In [ ]:
mu_M = mu_phi(phi_M)
mu_M

In [ ]:
sigma_M = sigma_phi(phi_M)
sigma_M

In [ ]:
r = 0.0025

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(frontier[:, 0], frontier[:, 1], 'm.', ms=5,
         label='efficient frontier')
plt.plot(0, r, 'o', ms=9, label='risk-less asset')
plt.plot(sigma_M, mu_M, '^', ms=9, label='market portfolio')
plt.plot((0, 0.6), (r, r + ((mu_M - r) / sigma_M) * 0.6),
         'r', label='capital market line', lw=2.0)
plt.annotate('$(0, \\bar{r})$', (0, r), (-0.015, r + 0.01))
plt.annotate('$(\sigma_M, \mu_M)$', (sigma_M, mu_M),
             (sigma_M - 0.025, mu_M + 0.01))
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.legend();

In [ ]:
def U(p):
    mu, sigma = p
    return mu - 1 / 2 * (sigma ** 2 + mu ** 2)

In [ ]:
cons = {'type': 'eq',
        'fun': lambda p: p[0] - (r + (mu_M - r) / sigma_M * p[1])}

In [ ]:
opt = minimize(lambda p: -U(p), (0.1, 0.3), constraints=cons)

In [ ]:
opt

In [ ]:
from sympy import *
init_printing(use_unicode=False, use_latex=False)

In [ ]:
mu, sigma, b, v = symbols('mu sigma b v')

In [ ]:
sol = solve('mu - b / 2 * (sigma ** 2 + mu ** 2) - v', mu)

In [ ]:
sol

In [ ]:
u1 = sol[0].subs({'b': 1, 'v': 0.1})
u1

In [ ]:
u2 = sol[0].subs({'b': 1, 'v': 0.125})
u2

In [ ]:
f1 = lambdify(sigma, u1)
f2 = lambdify(sigma, u2)

In [ ]:
sigma_ = np.linspace(0.0, 0.5)
u1_ = f1(sigma_)
u2_ = f2(sigma_)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(sigma_, u1_, label='$v=0.1$')
plt.plot(sigma_, u2_, '--', label='$v=0.125$')
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.legend();

In [ ]:
u = sol[0].subs({'b': 1, 'v': -opt['fun']})
u

In [ ]:
f = lambdify(sigma, u)

In [ ]:
u_ = f(sigma_)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(0, r, 'o', ms=9, label='risk-less asset')
plt.plot(sigma_M, mu_M, '^', ms=9, label='market portfolio')
plt.plot(opt['x'][1], opt['x'][0], 'v', ms=9, label='optimal portfolio')
plt.plot((0, 0.5), (r, r + (mu_M - r) / sigma_M * 0.5),
         label='capital market line', lw=2.0)
plt.plot(sigma_, u_, '--', label='$v={}$'.format(-round(opt['fun'], 3)))
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.legend();

## Arbitrage Pricing Theory

In [ ]:
M1

In [ ]:
M0

In [ ]:
V1 = np.array((12, 15, 7))

In [ ]:
reg = np.linalg.lstsq(M1, V1, rcond=-1)[0]
reg

In [ ]:
np.dot(M1, reg)

In [ ]:
np.dot(M1, reg) - V1

In [ ]:
V0 = np.dot(M0, reg)
V0

In [ ]:
U0 = 10
U1 = np.array((12, 5, 11))

In [ ]:
M0_ = np.array((S0, T0, U0))

In [ ]:
M1_ = np.concatenate((M1.T, np.array([U1,]))).T

In [ ]:
M1_

In [ ]:
np.linalg.matrix_rank(M1_)

In [ ]:
reg = np.linalg.lstsq(M1_, V1, rcond=-1)[0]
reg

In [ ]:
np.allclose(np.dot(M1_, reg), V1)

In [ ]:
V0_ = np.dot(M0_, reg)
V0_

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

<br><br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>